# Project 3
### Aileen Yang cy2830

**Dataset(s) to be used:** <br>
1. Counts and rates of asthma emergency department visits by zip code, California
https://healthdata.gov/State/Asthma-Emergency-Department-Visit-Rates/28nb-65xq/about_data 

2. California FIRE Damage Inspection Data
https://hub-calfire-forestry.hub.arcgis.com/datasets/cal-fire-damage-inspection-dins-data/about <br>


**Analysis question:** Do California areas that experience more wildfire activity have higher rates of asthma emergency department visits in children in the same year?

**Columns that will (likely) be used:**

From asthma dataset:
- Year
- Zip Code
- County
- Age_Group
- Number_of_Asthma_ED_Visits
- Age_Adjusted_Rate_of_Asthma_ED_V

From wildfire dataset:
- OBJECTID (unique structure record)
- Incident Name
- Incident Start Date
- Zip Code

**Columns to be used to merge/join them:**
  -  Asthma: Year, Zip Code
  - Wildfire: Year (from Incident Start Date), Zip Code

**Hypothesis**: California ZIP codes with more wildfire activity in a given year will have higher age-adjusted rates of asthma ED visits, particularly among children. 

In [35]:
import pandas as pd
import numpy as np
import plotly.express as px

**Cleansing of Asthma Data**

In [36]:
asthma = pd.read_csv('asthmaedvisitrates-by-zipcode.csv')
asthma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19190 entries, 0 to 19189
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Year                              19190 non-null  int64  
 1   Zip_Code                          19190 non-null  int64  
 2   County                            19190 non-null  object 
 3   Age_Group                         19190 non-null  object 
 4   Number_of_Asthma_ED_Visits        19190 non-null  int64  
 5   Age_Adjusted_Rate_of_Asthma_ED_V  19190 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 899.7+ KB


In [37]:
asthma = asthma.rename(columns={'Zip_Code':'Zip Code'})
asthma["Zip Code"] = asthma["Zip Code"].astype(str)

# Rename Columns
asthma = asthma.rename(columns={'Age_Adjusted_Rate_of_Asthma_ED_V': 'Age-adjusted Rate'})
asthma = asthma.rename(columns={'Number_of_Asthma_ED_Visits': 'Number of ED visits'})

In [ ]:
asthma["County"] = asthma["County"].astype(str).str.strip()
asthma = asthma.dropna(subset=['Year', 'County']).copy()

In [39]:
# Aggregate ed visits for each county
asthma = (
    asthma
    .groupby(["Year", "County"])
    .agg(
        total_ed_visits   = ("Number of ED visits", "sum"),
        mean_age_adj_rate = ("Age-adjusted Rate", "mean"),
    )
    .reset_index()
)

asthma.head()


,Year,County,total_ed_visits,mean_age_adj_rate
0,2013,Alameda,9866,76.067059
1,2013,Amador,207,76.257143
2,2013,Butte,906,53.550000
3,2013,Calaveras,117,87.666667
4,2013,Colusa,54,64.500000


**Importing Wildfire Dataset**

In [40]:
wildfire = pd.read_csv('Cali_Fire.csv')
wildfire.head()

/var/folders/sg/bpgt4bnn1qbfgvlc96v1x2m00000gn/T/ipykernel_1446/1965711151.py:1: DtypeWarning:

Columns (12,36,37) have mixed types. Specify dtype option on import or set low_memory=False.



,OBJECTID,* Damage,* Street Number,* Street Name,"* Street Type (e.g. road, drive, lane, etc.)","Street Suffix (e.g. apt. 23, blding C)",* City,State,Zip Code,* CAL FIRE Unit,...,Fire Name (Secondary),APN (parcel),Assessed Improved Value (parcel),Year Built (parcel),Site Address (parcel),GLOBALID,Latitude,Longitude,x,y
0,1,No Damage,8376.0,Quail Canyon,Road,NaN,Winters,CA,NaN,LNU,...,Quail,0101090290,510000.0,1997.0,8376 QUAIL CANYON RD VACAVILLE CA 95688,e1919a06-b4c6-476d-99e5-f0b45b070de8,38.474960,-122.044465,-1.358593e+07,4.646741e+06
1,2,Affected (1-9%),8402.0,Quail Canyon,Road,NaN,Winters,CA,NaN,LNU,...,Quail,0101090270,573052.0,1980.0,8402 QUAIL CANYON RD VACAVILLE CA 95688,b090eeb6-5b18-421e-9723-af7c9144587c,38.477442,-122.043252,-1.358579e+07,4.647094e+06
2,3,No Damage,8430.0,Quail Canyon,Road,NaN,Winters,CA,NaN,LNU,...,Quail,0101090310,350151.0,2004.0,8430 QUAIL CANYON RD VACAVILLE CA 95688,268da70b-753f-46aa-8fb1-327099337395,38.479358,-122.044585,-1.358594e+07,4.647366e+06
3,4,No Damage,3838.0,Putah Creek,Road,NaN,Winters,CA,NaN,LNU,...,Quail,0103010240,134880.0,1981.0,3838 PUTAH CREEK RD WINTERS CA 95694,64d4a278-5ee9-414a-8bf4-247c5b5c60f9,38.487313,-122.015115,-1.358266e+07,4.648497e+06
4,5,No Damage,3830.0,Putah Creek,Road,NaN,Winters,CA,NaN,LNU,...,Quail,0103010220,346648.0,1980.0,3830 PUTAH CREEK RD WINTERS CA 95694,1b44b214-01fd-4f06-b764-eb42a1ec93d7,38.485636,-122.016122,-1.358277e+07,4.648259e+06


In [41]:
wildfire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130722 entries, 0 to 130721
Data columns (total 46 columns):
 #   Column                                                        Non-Null Count   Dtype  
---  ------                                                        --------------   -----  
 0   OBJECTID                                                      130722 non-null  int64  
 1   * Damage                                                      130722 non-null  object 
 2   * Street Number                                               126302 non-null  float64
 3   * Street Name                                                 125236 non-null  object 
 4   * Street Type (e.g. road, drive, lane, etc.)                  116260 non-null  object 
 5   Street Suffix (e.g. apt. 23, blding C)                        62017 non-null   object 
 6   * City                                                        98991 non-null   object 
 7   State                                                   

In [42]:
# Drop noise columns
wildfire = wildfire.rename(columns = {'* Incident Name':'Incident Name'})
wildfire = wildfire[["OBJECTID", "Incident Name", "Incident Start Date", "County"]].copy()

# Extract year to match the format of asthma years
wildfire["Incident Start Date"] = pd.to_datetime(
    wildfire["Incident Start Date"], errors="coerce"
)
wildfire = wildfire.dropna(subset=["Incident Start Date"])
wildfire["Year"] = wildfire["Incident Start Date"].dt.year.astype(int)

# Clean County column
wildfire["County"] = wildfire["County"].astype(str).str.strip().str.title()

# Extract year
wildfire['Incident Start Date'] = pd.to_datetime(
    wildfire['Incident Start Date'], errors="coerce"
)
wildfire["Year"] = wildfire["Incident Start Date"].dt.year

wildfire.head()

/var/folders/sg/bpgt4bnn1qbfgvlc96v1x2m00000gn/T/ipykernel_1446/3929528454.py:6: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,OBJECTID,Incident Name,Incident Start Date,County,Year
0,1,Quail,2020-06-06,Solano,2020
1,2,Quail,2020-06-06,Solano,2020
2,3,Quail,2020-06-06,Solano,2020
3,4,Quail,2020-06-06,Solano,2020
4,5,Quail,2020-06-06,Solano,2020


In [43]:
wildfire = (
    wildfire
    .groupby(["Year", "County"])
    .agg(
        wildfire_records=("OBJECTID", "count"),
        wildfire_incidents=("Incident Name", "nunique")
    )
    .reset_index()
)

wildfire.head()

,Year,County,wildfire_records,wildfire_incidents
0,2013,Riverside,58,1
1,2013,Shasta,222,1
2,2014,Mendocino,11,1
3,2014,San Diego,49,1
4,2014,Siskiyou,250,1


**Merge the two Datasets**

In [44]:
merged = asthma.merge(
    wildfire,
    how="left",
    on=["Year", "County"]
)

# If there are no wildfire incidents, replace the value by 0
merged["wildfire_records"] = merged["wildfire_records"].fillna(0).astype(int)
merged["wildfire_incidents"] = merged["wildfire_incidents"].fillna(0).astype(int)
merged.tail(10)


,Year,County,total_ed_visits,mean_age_adj_rate,wildfire_records,wildfire_incidents
547,2022,Sonoma,1027,30.180769,0,0
548,2022,Stanislaus,2078,46.642857,0,0
549,2022,Sutter,230,28.680000,0,0
550,2022,Tehama,173,43.240000,51,4
551,2022,Trinity,17,69.500000,0,0
552,2022,Tulare,1345,42.339130,0,0
553,2022,Tuolumne,68,43.600000,7,2
554,2022,Ventura,1876,30.470270,0,0
555,2022,Yolo,540,35.266667,0,0
556,2022,Yuba,283,46.025000,15,1


In [45]:
# Categorize exposures according to the occurrence
def exposure_category(n):
    if n == 0:
        return "No wildfire"
    elif n < 20:
        return "Low wildfire (1–19)"
    elif n < 100:
        return "Moderate wildfire (20–99)"
    else:
        return "High wildfire (100+)"
        
merged["exposure_cat"] = merged["wildfire_records"].apply(exposure_category)
merged["exposure_cat"].value_counts()


exposure_cat
No wildfire                  394
High wildfire (100+)          70
Low wildfire (1–19)           60
Moderate wildfire (20–99)     33
Name: count, dtype: int64

In [46]:
summary = (
    merged.groupby("exposure_cat")
    .agg(
        counties=("County", "nunique"),
        mean_rate=("mean_age_adj_rate", "mean"),
        median_rate=("mean_age_adj_rate", "median"),
        mean_total_ed=("total_ed_visits", "mean"),
        max_wildfire=("wildfire_records", "max")
    )
    .reset_index()
    .sort_values("mean_rate", ascending=False)
)

summary


,exposure_cat,counties,mean_rate,median_rate,mean_total_ed,max_wildfire
3,No wildfire,56,53.528225,51.738478,2612.670051,0
1,Low wildfire (1–19),36,49.194723,46.085776,1810.166667,19
2,Moderate wildfire (20–99),25,42.486158,41.125000,2048.515152,95
0,High wildfire (100+),35,41.446272,37.836364,4002.814286,23625


In [ ]:
fig = px.scatter(
    merged,
    x="wildfire_records",
    y="mean_age_adj_rate",
    color="exposure_cat",
    hover_data=["County", "Year", "total_ed_visits"],
    title="County-Level Asthma ED Rate vs Wildfire Activity",
    labels={
        "wildfire_records": "Wildfire structure records (CAL FIRE, County × Year)",
        "mean_age_adj_rate": "Mean age-adjusted asthma ED rate"
    }
)
fig.show()


In [48]:
trend = (
    merged.groupby(["Year", "exposure_cat"])
    .agg(mean_rate=("mean_age_adj_rate", "mean"))
    .reset_index()
)

fig = px.line(
    trend,
    x="Year",
    y="mean_rate",
    color="exposure_cat",
    markers=True,
    title="Asthma ED Rates Over Time by Wildfire Exposure (County-level)",
)
fig.show()

**For Children**